# 优化模型的参数
前面已经建立好了模型，有许多的参数需要通过训练、验证、测试来进行优化。训练一个模型是一个迭代化的进程，每一次迭代(在PyTorch中叫做`epoch`)，我们的模型都会猜测相应的输出，计算猜测的输出与真实的输出之间的差距(`loss`),记录这些参数的导数，然后通过梯度下降优化这些参数。

## 预备代码
在进行模型的训练和优化之前，需要先完成**数据集创建和加载**，**模型的创建**

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# 数据集加载
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 模型创建
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 超参数
超参数是我们用来控制迭代进程的一类参数值，不同的参数可以用来影响模型的训练和控制模型的收敛速度。在PyTorch中定义了如下的参数。
+ Number Of Epochs: 迭代数据集的代数或者次数
+ Batch Size: 在参数更新前一次性加载进模型的数量
+ Learning Rate: 控制更新模型的参数的速度。小的值表示的就是较低的学习率，但是值太大的话也可能带来意想不到的结果。

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 优化迭代循环
当我们设置完超参数，那我们可以在优化循环中对我们的模型进行训练和优化，每一次的优化迭代循环都叫做一个`epoch`,每一个`epoch`都包含两部分。
+ Train Loop: 在训练集上的迭代次数，目标时收敛到最佳参数
+ Test/Validation Loop：在测试或验证集上的迭代次数，目标时验证模型的效果是否得到改善。

# 损失函数
当仅仅给模型输入一些训练数据时，模型给出的结果似乎时不正确的。**损失函数**度量得到的结果与目标的不相似程度，这也是我们在训练过程中想最小化的一个函数。为了计算损失，我们输入给模型输入一些数据，得到预测结果，并将预测的结果与真实的标签进行比照。

常见的损失函数有
+ 均方误差(Mean Square Error,MSE, `nn.MSELoss`),用于回归任务
+ 负对数似然(Negative Log Likelihood, `NLL,nn.NLLLose`),用于分类认任务
+ 交叉商损失(CrossEntropyLoss, `nn.CrossEntropyLoss`)，结合了负对数似然和`Softmax`函数，用于多分类


In [3]:
loss_fn = nn.CrossEntropyLoss()

# 优化器
优化器就是在每次的迭代过程中，优化模型的参数，减小模型的损失的一个过程。优化算法定义了这个进程时如何实现和运行的。
所有的优化逻辑都要封装在`optimizer`这个对象中，本示例中我们使用的是随机梯度下降算法(Stochastic Gradient Descent, SGD)优化器，在PyTorch中，定义了许多的优化器，这些优化器在不同的模型或者参数上能够表现得更好。
建立一个优化器，需要传递**模型的参数**和**学习率**两个主要的参数。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化器一般会进行以下三个步骤。
1. 调用`optimizer.zero_grad()`，重置模型参数的梯度。默认情况下，模型参数的梯度时会累加的，因此需要在每次迭代前将其置为0
2. 反向传播预测的损失，也就是调用`loss.backwards()`,PyTorch记录了各个参数反向传播的梯度。
3. 当我们计算得到了梯度，将调用`optimizer.step()`,来通过损失的梯度来优化调整模型的参数。

## 优化器实现完整

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    print(dataloader)
    size = len(dataloader.dataset) # 获取整个数据集的大小
    for batch, (X, y) in enumerate(dataloader): # 循环获取每一个batch
    # 此处的batch是迭代加载的次数，每一次加载进来64张图片，然后64张图片进行一次训练，然后进行下一次
    # 又是一样的加载64张图片进行训练
        # Compute prediction and loss
        print(batch,X,y)
        pred = model(X)  # 前向传播预测
        loss = loss_fn(pred, y)  # 计算损失

        # Backpropagation
        optimizer.zero_grad()  # 反向传播之前梯度清零
        loss.backward()  # 反向传播
        optimizer.step()  # 优化器优化

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)   # 实际运行的个数是需要迭代次数乘以每一批次的大小的
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

初始化损失函数和优化器，然后传递给上面的两个函数。

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

流式输出内容被截断，只能显示最后 5000 行内容。
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0157, 0.0000],
          ...,
          [0.0000, 0.0627, 0.3922,  ..., 0.5255, 0.8745, 0.0000],
          [0.0039, 0.0000, 0.0000,  ..., 0.0000, 0.2000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        ...,


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0118, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.4314, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.4118, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.1333, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000